# Project 2 Risk Parity

# In this part, our asset groups include one gold ETF named GLD and one stock  ETF tracking the NASDAQUE 100 named QQQ.
# We are interested in gold since we guess that the gold price should be stable across the time. Bond might be turned into junk bond while the gold could not be downgraded.
# The top holdings of the QQQ ETF are all technology stocks which we are interested in.
# The market cap for these two ETFs are similar to some extent.
# The time horizon of our dataset starts from 2007-12-31 to 2017-12-31

In [ ]:
# In this part, we define some cornerstones of this project such as optimization function for risk budgeting approach
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize

from pandas_datareader import data as pdr
import fix_yahoo_finance as yf
yf.pdr_override() 
import datetime 
from collections import OrderedDict
from datetime import timedelta

import warnings
warnings.filterwarnings("ignore")


def cov_ewma(ret_assets, lamda = 0.94):
    ret_mat = ret_assets.values
    T = len(ret_assets)
    coeff = np.zeros((T,1))
    S = ret_assets.cov()
    for i in range(1, T):
#        S = lamda * S  + (1-lamda)*np.matmul(ret_mat[i-1,:].reshape((-1,1)), 
#                          ret_mat[i-1,:].reshape((1,-1)))
        S = lamda * S  + (1-lamda)* (ret_mat[i-1,:].reshape((-1,1)) @ ret_mat[i-1,:].reshape((1,-1)) )
        
        coeff[i] = (1-lamda)*lamda**(i)
    return S/np.sum(coeff)

    
# risk budgeting approach optimisation object function
def obj_fun(W, cov_assets, risk_budget):
    var_p = np.dot(W.transpose(), np.dot(cov_assets, W))
    sigma_p = np.sqrt(var_p)
    risk_contribution = W*np.dot(cov_assets, W)/sigma_p
    risk_contribution_percent = risk_contribution/sigma_p
    return np.sum((risk_contribution_percent-risk_budget)**2)


# calculate risk budgeting portfolio weight give risk budget
def riskparity_opt(ret_assets, risk_budget, lamda, method='ewma',Wts_min=0.0, leverage=False):
    # number of assets
    num_assets = ret_assets.shape[1]
    # covariance matrix of asset returns
    if method=='ewma':
        cov_assets = cov_ewma(ret_assets, lamda)
    elif method=='ma':
        cov_assets = ret_assets.cov()
    else:
        cov_assets = cov_ewma(ret_assets, lamda)        
    
    # initial weights
    w0 = 1.0 * np.ones((num_assets, 1)) / num_assets
    # constraints
    #cons = ({'type': 'eq', 'fun': cons_sum_weight}, {'type': 'ineq', 'fun': cons_long_only_weight})
    if leverage == True:
        c_ = ({'type':'eq', 'fun': lambda W: sum(W)-2. }, # Sum of weights = 200%
              {'type':'ineq', 'fun': lambda W: W-Wts_min}) # weights greater than min wts
    else:
        c_ = ({'type':'eq', 'fun': lambda W: sum(W)-1. }, # Sum of weights = 100%
              {'type':'ineq', 'fun': lambda W: W-Wts_min}) # weights greater than min wts
    # portfolio optimisation
    return minimize(obj_fun, w0, args=(cov_assets, risk_budget), method='SLSQP', constraints=c_)


# function to get the price data from yahoo finance 
def getDataBatch(tickers, startdate, enddate):
  def getData(ticker):
    return (pdr.get_data_yahoo(ticker, start=startdate, end=enddate))
  datas = map(getData, tickers)
  return(pd.concat(datas, keys=tickers, names=['Ticker', 'Date']))


# function to get the return data calculated from price data 
# retrived from yahoo finance 
def getReturns(tickers, start_dt, end_dt, freq='monthly'): 
    px_data = getDataBatch(tickers, start_dt, end_dt)
    # Isolate the `Adj Close` values and transform the DataFrame
    px = px_data[['Adj Close']].reset_index().pivot(index='Date', 
                           columns='Ticker', values='Adj Close')
    if (freq=='monthly'):
        px = px.resample('M').last()
        
    # Calculate the daily/monthly percentage change
    ret = px.pct_change().dropna()
    
    ret.columns = tickers
    return(ret)

In [ ]:
# get historical price data of our two groups of assets
Flag_downloadData = False
# define the time period 
start_dt = datetime.datetime(2007, 12, 31)
end_dt = datetime.datetime(2017, 12, 31)

if Flag_downloadData:
#    price_SPX = pdr.get_data_yahoo('SPY', start=start_dt, end=end_dt)
#    price_AGG = pdr.get_data_yahoo('AGG', start=start_dt, end=end_dt)
    #
    Ticker_AllAsset = ['QQQ', 'GLD']
    stock_data = getDataBatch(Ticker_AllAsset, start_dt, end_dt)
    # Isolate the `Adj Close` values and transform the DataFrame
    price_AllAsset = stock_data.reset_index().pivot(index='Date', columns='Ticker', values='Adj Close')
    # Merge equity data with bond data
    # merging/joining dataframe
#    frames = [price_SPX['Adj Close'], price_AGG['Adj Close']]# this creats a list of two frames
#    price_AllAsset1 = pd.concat(frames, axis=1, join='inner') # merge only the common rows
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    writer = pd.ExcelWriter('QQQ-GLD.xlsx', engine='xlsxwriter')
    price_AllAsset.to_excel(writer, sheet_name='Price',startrow=0, startcol=0, header=True, index=True)
else:
    price_AllAsset = pd.read_excel('QQQ-GLD.xlsx', sheet_name='Price',
                    header=0, index_col = 0)

# In this part, we decide to use the window equal to 30 instead of 90
# The current running result allows for the shorting activity

In [ ]:
# 2. Calculate ARP excess returns
ret_assets = price_AllAsset.pct_change().dropna()
ret_assets_demean = ret_assets - ret_assets.mean()
num_assets = ret_assets.shape[1]

lamda = 0.94
SS = cov_ewma(ret_assets_demean, lamda)

# Construct risk parity portfolio
# portfolio dates - this defines the first date of portfolio construction
datestr = ret_assets.index[ret_assets.index >= '2008-03-31']
# previous month
mth_previous = datestr[0]
# initialise portfolio weights matrix
wts = pd.DataFrame(index=datestr, columns=ret_assets.columns)
# initialise portfolio return matrix
ret_riskParity = pd.DataFrame(index=datestr, columns=['Risk Parity'])
# how many rolling calendar days to use for covariance calculation
window = 30
Wts_min = 0.0
risk_budget = 1.0/num_assets*np.ones([1,num_assets]) #risk-party
#risk_budget = [0.7, 0.4]
leverage = True
varmodel = 'ewma'


for t in datestr:
    # construct risk budgeting portfolio and re-balance on monthly basis
    if t.month==mth_previous:
        # keep the same portfolio weights within the month
        wts.loc[t] = wts.iloc[wts.index.get_loc(t)-1]
    else:
        # update the value of the previous month 
        mth_previous = t.month
        # re-balance the portfolio at the start of the month
        
        t_begin = t - timedelta(days=window)
        ret_used = ret_assets.loc[t_begin:t,:]
        wts.loc[t] = riskparity_opt(ret_used, risk_budget, lamda, varmodel, Wts_min, leverage).x
    # calculate risk budgeting portfolio returns
    ret_riskParity.loc[t] = np.sum(wts.loc[t] * ret_assets.loc[t])
    
# Due to precision issue, wts could be a tiny negative number instead of zero, make them zero
wts[wts<0]=0.0
# Construct equal weighted portfolio
ret_equalwted = pd.DataFrame(np.sum(1.0*ret_assets[ret_assets.index>=datestr[0]]/num_assets, axis=1), columns=['Equal Weighted'])
# Construct 60/40 weighted portfolio
#ret_equalwted = pd.DataFrame(np.sum(1.0*ret_assets[ret_assets.index>=datestr[0]]/num_assets, axis=1), columns=['Equal Weighted'])

In [ ]:
# Calculate performance stats such as sharpe ratio and the cumulative returns
ret_cumu_assets = (ret_assets + 1).cumprod()
ret_cumu_riskP = (ret_riskParity + 1).cumprod()
ret_cumu_equalwt = (ret_equalwted + 1).cumprod()

ret_annual_assets = ret_cumu_assets.iloc[-1]**(250/len(ret_cumu_assets))-1
std_annual_assets = ret_assets.std()*np.sqrt(250)
sharpe_ratio_assets = ret_annual_assets/std_annual_assets

ret_annual_riskP = ret_cumu_riskP.iloc[-1]**(250/len(ret_cumu_riskP))-1
std_annual_riskP = ret_riskParity.std()*np.sqrt(250)
sharpe_ratio_riskP = ret_annual_riskP/std_annual_riskP

ret_annual_equalwt = ret_cumu_equalwt.iloc[-1]**(250/len(ret_cumu_equalwt))-1
std_annual_equalwt = ret_equalwted.std()*np.sqrt(250)
sharpe_ratio_equalwt = ret_annual_equalwt/std_annual_equalwt

#sharpe_table = [sharpe_ratio_riskP, sharpe_ratio_equalwt]
sharpe_table = pd.Series(OrderedDict((('risk_parity', sharpe_ratio_riskP.values),
                 ('equal_wted', sharpe_ratio_equalwt.values),
                 )))
sharpe_table1 = pd.Series(OrderedDict((('risk_parity', sharpe_ratio_riskP.values),
                 ('QQQ', sharpe_ratio_assets[0]),
                 ('GLD', sharpe_ratio_assets[1]),
                 )))
print('sharpe ratio of different strategies:\n',sharpe_table)
print('\nsharpe ratio of strategies vs assets:\n',sharpe_table1)

In [ ]:
# compare the portfolio cumulative returns based on whether short sale could be used
# In this part, we use the plot to better illustrate the results
if leverage:
        figure_count = 1
        plt.figure(figure_count)
        figure_count = figure_count+1
        pd.concat([ret_cumu_riskP, ret_cumu_equalwt], axis=1).plot()
        plt.ylabel('Cumulative Return')
        plt.title('Leverage cumulative return for different strategies',fontsize=10)
        plt.show()
        plt.savefig('Cumulative returns for different strategies_leverage')
        # compare the portfolio cumulative returns vs. asset returns
        plt.figure(figure_count)
        figure_count = figure_count+1
        pd.concat([ret_cumu_riskP, ret_cumu_assets], axis=1).plot()
        plt.ylabel('Cumulative Return')
        plt.title('Leverage returns between assets',fontsize=10)
        plt.show()
        plt.savefig('Cumulative returns for different strategies_leverage')
        # plot the historical weights of the assets
        # area plot showing the weights
        plt.figure(figure_count)
        figure_count = figure_count + 1
        wts.plot.area()
        plt.title('Leverage asset weights',fontsize=10)
        plt.ylabel('asset weights')
        plt.savefig('leverage asset weights')
else:
    
    figure_count = 1
    plt.figure(figure_count)
    figure_count = figure_count+1
    pd.concat([ret_cumu_riskP, ret_cumu_equalwt], axis=1).plot()
    plt.ylabel('Cumulative Return')
    plt.title('Cumulative return for different strategies',fontsize=10)
    plt.show()
    plt.savefig('Cumulative returns for different strategies')
    # compare the portfolio cumulative returns vs. asset returns
    plt.figure(figure_count)
    figure_count = figure_count+1
    pd.concat([ret_cumu_riskP, ret_cumu_assets], axis=1).plot()
    plt.ylabel('Cumulative Return')
    plt.title('Returns between assets',fontsize=10)
    plt.show()
    plt.savefig('Cumulative returns for different strategies')
    # plot the historical weights of the assets
    # area plot showing the weights
    plt.figure(figure_count)
    figure_count = figure_count + 1
    wts.plot.area()
    plt.title('Asset weights',fontsize=10)
    plt.ylabel('asset weights')
    plt.savefig('Asset weights')

# Some insights from the results
# No matter the shorting is allowed, the sharpe ratio of the risk parity is always higher than that of the equally weighted strategy and each asset.
# Therefore, the risk parity strategy is more optimal than equally weighted strategy from the perspective of risk budgeting 
# The sharpe ratio of the risk parity is higher when the shorting is not allowed. We guess that the reason for this could be: the price trend of these two assets has relative big difference, sometimes exists negative relationship. Therefore, when we use the more risky shorting investment strategy, the excess return we could get fails to compensate the increased risk we have taken.
# When comparing the cumulative return, risk parity beats the equally weighted strategy whenever using the shortings.
# The trade-off decision should depend on the degree of investor's risk aversion. If you are a risk-lover, risk parity stratgy with short sales will be definitely the best choice for you. If you are risk-averse, you definitely focus more on the excess return per unit of risk. In this case, you should choose the risk parity strategy without short sales which has the highest sharpe ratio

# More to think about:
# How to choose the assets groups to get a higher sharpe ratio using the risk parity strategy?
# Is it more optimal to use the risk parity strategy when choosing asset groups with higher correlation?